In [1]:
import pandas as pd
import re

Loading Data

In [2]:
df = pd.read_csv('../parse_phones/phones.csv')
cdf = pd.DataFrame() # CDF - Clean Data Frame
cdf[['brand', 'model']] = df['name'].str.split(' ', n=1, expand=True)

/tmp/ipykernel_30915/2167149916.py:1: DtypeWarning: Columns (96,100,101,103,104,105) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../parse_phones/phones.csv')


In [3]:
def extract_euro_price(price_string):
    # Conversion rates for the date of extraction
    INR_TO_EUR_2025_04_14 = 0.01029
    USD_TO_EUR_2025_04_14 = 0.8791
    GBP_TO_EUR_2025_04_14 = 1.167

    # If value is na return None
    if pd.isna(price_string) or not isinstance(price_string, str):
        return None
    
    euro_pattern = r'€\s*([\d,]+\.?\d{2})|\b(\d+\.?\d*)\s*EUR|₹\s*([\d,]+\.?\d{2})|\b(\d+\.?\d*)\s*INR|\$\s*([\d,]+\.?\d{2})|£\s*([\d,]+\.?\d{2})'
    match = re.search(euro_pattern, price_string)
    if match:
        # Check for each group in the order of priority
        if match.group(1):  # Euro price with decimal
            return float(match.group(1).replace(',', ''))
        elif match.group(2):  # Euro price without currency
            return float(match.group(2).replace(',', ''))
        elif match.group(3):  # Indian Rupee price
            return float(match.group(3).replace(',', '')) * INR_TO_EUR_2025_04_14
        elif match.group(4):  # Indian Rupee price
            return float(match.group(4).replace(',', '')) * INR_TO_EUR_2025_04_14
        elif match.group(5):  # USD price
            return float(match.group(5).replace(',', '')) * USD_TO_EUR_2025_04_14
        elif match.group(6):  # British pound price
            return float(match.group(6).replace(',', '')) * GBP_TO_EUR_2025_04_14
    return None

Extracting features

In [4]:
# Links, Popularity, Price
cdf[['photo_link', 'phone_link', 'popularity_become_fan']] = df[['photo_link', 'phone_link', 'popularity_become_fan']]
cdf['popularity_views'] = pd.to_numeric(df['popularity_hits'].str.replace(',', '').str.replace(' hits', ''))
cdf['popularity_views_today'] = pd.to_numeric(df['popularity_percent'].str.extract(r'(\d+\.?\d*)%')[0])
cdf['price'] = df['Misc_Price'].apply(extract_euro_price)

# Network
cdf['network_technology'] = df['Network_Technology']
cdf['eSIM'] = df['Body_SIM'].str.contains(r'esim', case=False)

# Release Dates
cdf['announce_year'] = df['Launch_Announced'].str.extract(r'(\d{4})')
cdf['available'] = df['Launch_Status'].str.contains('Available', case=False)
cdf['release_year'] = df['release_date'].str.extract(r'(\d{4})')
cdf['cancelled'] = df['release_date'].str.contains(r'Cancelled', case=False)

# Physiscal Properties
cdf[['height_mm', 'length_mm', 'width_mm']] = df['Body_Dimensions'].str.extract(r'(\d+\.\d+|\d+)\s*m*\s*x\s*(\d+\.\d+|\d+)\s*m*\s*x\s*(\d+\.\d+|\d+)')
cdf['weight_g'] = df['Body_Weight'].str.extract(r'(\d+\.?\d+)')
cdf['ip_rating'] = df['Body_DummyType_'].str.extract(r'(IP[X]?\d{1,2}[X]?)')

# Extracting screen data
cdf['screen_type'] = df['Display_Type'].str.extract(r'(AMOLED|LCD|TFT|IPS|PLS|LTPO|OLED|CSTN|STN|TFD|FSTN|UFB|GCS|UBC|TN|Monochrome|Grayscale)')
cdf['screen_hz'] = df['Display_Type'].str.extract(r'(\d+\.?\d*)\s*Hz', flags=re.IGNORECASE)
cdf['screen_hz'] = cdf['screen_hz'].fillna(60) # Assume that standard refresh rate is 60 hertz
cdf['screen_size'] = df['Display_Size'].str.extract(r'(\d+\.?\d+)')
cdf['screen_to_body'] = df['Display_Size'].str.extract(r'(\d+\.?\d+)%')
cdf[['screen_resolution_x', 'screen_resolution_y']] = df['Display_Resolution'].str.extract(r'(\d+\.?\d*)\s*x?\s*(\d+\.?\d*)')

# Extracting os data
cdf['os_version'] = df['Platform_OS'].str.extract(r'([A-Za-z\s]+\s*\d*(\.\d+)*)')[0]

# Extracting Hardware data
cdf['chipset_nm'] = df['Platform_Chipset'].str.extract(r'(\d+\.?\d*)\s*nm')
cdf['chipset_cores'] = df['Platform_CPU'].str.extract(r'(\w+-?core)', flags=re.IGNORECASE)
cdf['gpu_model'] = df['Platform_GPU']

# Extracting memory data
cdf['memory_card_slot'] = ~df['Memory_Card slot'].str.contains(r'No', case=False)
cdf[['internal_rom', 'internal_rom_unit']] = df['Memory_Internal'].str.extract(r'(\d+\.?\d*)\s*(GB|MB|KB)(?!\s*RAM)(?:\s*ROM)?', flags=re.IGNORECASE)
cdf[['internal_ram', 'internal_ram_unit']] = df['Memory_Internal'].str.extract(r'(\d+\.?\d*)\s*(GB|MB|KB)(?:\s*RAM)(?!\s*ROM)', flags=re.IGNORECASE)

# Extracting camera data
cdf['camera_mp'] = df['Main Camera_Single'].str.extract(r'(\d+\.?\d*)\s*MP', flags=re.IGNORECASE)
cdf['camera_mp'] = cdf['camera_mp'].fillna(df['Main Camera_Dual'].str.extract(r'(\d+\.?\d*)\s*MP', flags=re.IGNORECASE)[0])
cdf['camera_mp'] = cdf['camera_mp'].fillna(df['Main Camera_Dual or Triple'].str.extract(r'(\d+\.?\d*)\s*MP', flags=re.IGNORECASE)[0])
cdf['camera_mp'] = cdf['camera_mp'].fillna(df['Main Camera_Five'].str.extract(r'(\d+\.?\d*)\s*MP', flags=re.IGNORECASE)[0])
cdf['camera_mp'] = cdf['camera_mp'].fillna(df['Main Camera_Penta'].str.extract(r'(\d+\.?\d*)\s*MP', flags=re.IGNORECASE)[0])
cdf['camera_mp'] = cdf['camera_mp'].fillna(df['Main Camera_Quad'].str.extract(r'(\d+\.?\d*)\s*MP', flags=re.IGNORECASE)[0])
cdf['camera_mp'] = cdf['camera_mp'].fillna(df['Main Camera_Triple'].str.extract(r'(\d+\.?\d*)\s*MP', flags=re.IGNORECASE)[0])
cdf['camera_f'] = df['Main Camera_Single'].str.extract(r'f/?\s*(\d+\.?\d*)', flags=re.IGNORECASE)
cdf['camera_f'] = cdf['camera_f'].fillna(df['Main Camera_Dual'].str.extract(r'f/?\s*(\d+\.?\d*)', flags=re.IGNORECASE)[0])
cdf['camera_f'] = cdf['camera_f'].fillna(df['Main Camera_Dual or Triple'].str.extract(r'f/?\s*(\d+\.?\d*)', flags=re.IGNORECASE)[0])
cdf['camera_f'] = cdf['camera_f'].fillna(df['Main Camera_Five'].str.extract(r'f/?\s*(\d+\.?\d*)', flags=re.IGNORECASE)[0])
cdf['camera_f'] = cdf['camera_f'].fillna(df['Main Camera_Penta'].str.extract(r'f/?\s*(\d+\.?\d*)', flags=re.IGNORECASE)[0])
cdf['camera_f'] = cdf['camera_f'].fillna(df['Main Camera_Quad'].str.extract(r'f/?\s*(\d+\.?\d*)', flags=re.IGNORECASE)[0])
cdf['camera_f'] = cdf['camera_f'].fillna(df['Main Camera_Triple'].str.extract(r'f/?\s*(\d+\.?\d*)', flags=re.IGNORECASE)[0])
cdf['camera_video_resolution'] = df['Main Camera_Video'].str.extract(r'(\d+\.?\d*\s*(?:K|p)|Yes|No|QCIF|QVGA|CIF|(?:(?:\d+\.?\d*)\s*x?\s*(?:\d+\.?\d*)))', flags=re.IGNORECASE)
cdf['camera_video_fps'] = df['Main Camera_Video'].str.extract(r'(\d+\.?\d*)\s*fps', flags=re.IGNORECASE)

# Selfie camera
cdf['selfie_camera_mp'] = df['Selfie camera_Single'].str.extract(r'(\d+\.?\d*)\s*MP|Yes', flags=re.IGNORECASE)
cdf['selfie_camera_mp'] = cdf['selfie_camera_mp'].fillna(df['Selfie camera_Dual'].str.extract(r'(\d+\.?\d*)\s*MP', flags=re.IGNORECASE)[0])
cdf['selfie_camera_mp'] = cdf['selfie_camera_mp'].fillna(df['Selfie camera_Triple'].str.extract(r'(\d+\.?\d*)\s*MP', flags=re.IGNORECASE)[0])
cdf['selfie_camera_f'] = df['Selfie camera_Single'].str.extract(r'f/?\s*(\d+\.?\d*)', flags=re.IGNORECASE)
cdf['selfie_camera_f'] = cdf['selfie_camera_f'].fillna(df['Selfie camera_Dual'].str.extract(r'f/?\s*(\d+\.?\d*)', flags=re.IGNORECASE)[0])
cdf['selfie_camera_f'] = cdf['selfie_camera_f'].fillna(df['Selfie camera_Triple'].str.extract(r'f/?\s*(\d+\.?\d*)', flags=re.IGNORECASE)[0])
cdf['selfie_camera_video_resolution'] = df['Selfie camera_Video'].str.extract(r'(\d+\.?\d*\s*(?:K|p)|Yes|No|QCIF|QVGA|CIF|(?:(?:\d+\.?\d*)\s*x?\s*(?:\d+\.?\d*)))', flags=re.IGNORECASE)
cdf['selfie_camera_video_fps'] = df['Selfie camera_Video'].str.extract(r'(\d+\.?\d*)\s*fps', flags=re.IGNORECASE)

# Sound
cdf['loudspeaker'] = ~df['Sound_Loudspeaker '].str.contains(r'No', case=False)
cdf['35mm_jack'] = ~df['Sound_3.5mm jack '].str.contains(r'No', case=False, na=False)

# Comms
cdf['wifi_model'] = df['Comms_WLAN'].str.extract(r'\d+\.?\d*\s*(\w+(?:/\w+)*)')
cdf['bluetooth_version'] = df['Comms_Bluetooth'].str.extract(r'(\d+\.?\d*)')
cdf['gps'] = df['Comms_Positioning'].str.contains(r'GPS', case=False, na=False)
cdf['nfc'] = ~df['Comms_NFC'].str.contains(r'No', case=False, na=True)
cdf['radio'] = ~df['Comms_Radio'].str.contains(r'No', case=False, na=True)
cdf['usb_type'] = df['Comms_USB'].str.extract(r'([a-zA-Z\-]*\s*USB\s*[a-zA-Z\-]*|Lightning)', flags=re.IGNORECASE)
cdf['usb_version'] = df['Comms_USB'].str.extract(r'(\d+\.?\d*)')

# Features
cdf['biometric_auth'] = df['Features_Sensors'].str.contains(r'Fingerprint|Face', case=False, na=False)
cdf['has_black_color'] = df['Misc_Colors'].str.contains(r'Black', case=False, na=False)
cdf['foldable'] = df['Display_Type'].str.contains('Foldable', case=False, na=False)

# Battery
cdf['battery_type'] = df['Battery_Type'].str.extract(r'(Li-Po|Li-Ion|NiMH)', flags=re.IGNORECASE)
cdf['battery_capacity'] = df['Battery_Type'].str.extract(r'(\d+\.?\d*)\s*mAh', flags=re.IGNORECASE)



In [7]:
missing = df[cdf['battery_type'].isna()] # Dataset to examine missing values which were not captured using regex

In [6]:
cdf.to_csv('extracted_phones.csv', index=False)